ETL Project: Beer Production Data Base

Team Members:

- Esther Villarreal
- Daniel Orta
- Mauricio Arroyo

In [1]:
# Import Dependencies

import pandas as pd
from sqlalchemy import create_engine

We obtained this dataset from Kaggle.com comes from the Alcohol and Tobacco Tax and Trade Bureau (TTB). 

We used 2 csv files from this dataset, beer_states.csv and brewing_materials.csv

https://www.kaggle.com/jessemostipak/beer-production

In [2]:
beer_states_file = "Resources/beer_states.csv"
beer_states_df = pd.read_csv(beer_states_file)
beer_states_df.head()

,state,year,barrels,type
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises


In [3]:
brewing_materials_file = "Resources/brewing_materials.csv"
brewing_materials_df = pd.read_csv(brewing_materials_file)
brewing_materials_df.head()

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
0,Pounds of Materials Used,Grain Products,2008,1,Malt and malt products,374165152,365300134,374165152.0,365300134.0
1,Pounds of Materials Used,Grain Products,2008,1,Corn and corn products,57563519,41647092,57563519.0,41647092.0
2,Pounds of Materials Used,Grain Products,2008,1,Rice and rice products,72402143,81050102,72402143.0,81050102.0
3,Pounds of Materials Used,Grain Products,2008,1,Barley and barley products,3800844,2362162,3800844.0,2362162.0
4,Pounds of Materials Used,Grain Products,2008,1,Wheat and wheat products,1177186,1195381,1177186.0,1195381.0


Since our data was already processed, we had to reverse engineer the data to get smaller dataframes and eliminate the rows that contained the total values of the information so we could obtain it ourselves.

The cleaning of the data consisted of renaming the columns to make it more understandable, to facilitate the merging of the dataframes and be able to connect it to the sql server.

In [4]:
beer_states_df = beer_states_df[beer_states_df["state"].str.contains("total")==False]
beer_states_df = beer_states_df.rename(columns={"type":"presentation"})

presentations = beer_states_df.presentation.unique()
presentations_df = pd.DataFrame(presentations)
presentations_df = presentations_df.rename(columns={0:"presentation"})
beer_states_df

,state,year,barrels,presentation
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises
5,AK,2013,2155.60,On Premises
6,AK,2014,2114.97,On Premises
7,AK,2015,4511.48,On Premises
8,AK,2016,13315.93,On Premises
9,AK,2017,15974.10,On Premises


In [5]:
brewing_materials_df = brewing_materials_df[['material_type', 'year', 'month', 'type','month_current']]
brewing_materials_df = brewing_materials_df[brewing_materials_df["material_type"].str.contains("Total")==False]
brewing_materials_df.head()

,material_type,year,month,type,month_current
0,Grain Products,2008,1,Malt and malt products,374165152
1,Grain Products,2008,1,Corn and corn products,57563519
2,Grain Products,2008,1,Rice and rice products,72402143
3,Grain Products,2008,1,Barley and barley products,3800844
4,Grain Products,2008,1,Wheat and wheat products,1177186


In [6]:
material_brewing = brewing_materials_df.material_type.unique()
material_brewing_df = pd.DataFrame(material_brewing)
material_brewing_df = material_brewing_df.rename(columns={0:"material_type"})
material_brewing_df

,material_type
0,Grain Products
1,Non-Grain Products


In [7]:
types_brewing = brewing_materials_df.type.unique()
types_brewing_df = pd.DataFrame(types_brewing)
types_brewing_df = types_brewing_df.rename(columns={0:"type"})
types_brewing_df

,type
0,Malt and malt products
1,Corn and corn products
2,Rice and rice products
3,Barley and barley products
4,Wheat and wheat products
5,Sugar and syrups
6,Hops (dry)
7,Hops (used as extracts)
8,Other


Created the connection to Postgres

In [8]:
rds_connection_string = "postgres:nopal91@localhost:5432/beer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Confirmed the tables

In [9]:
engine.table_names()

['material_brewing',
 'types_brewing',
 'presentation',
 'merged_brewing_materials',
 'merged_beer_states']

Loaded the dataframes to the data base.

In [10]:
material_brewing_df.to_sql(name="material_brewing", con=engine, if_exists="append", index=False)

In [11]:
types_brewing_df.to_sql(name="types_brewing", con=engine, if_exists="append", index=False)

In [12]:
presentations_df.to_sql(name="presentation", con=engine, if_exists="append", index=False)

Retrieved the data from sql to obtain the id from each table.

In [13]:
#get the DataFrame from sql with the id created from the data base.
types_brewing_df = pd.read_sql_query('select * from types_brewing', con=engine)
material_brewing_df = pd.read_sql_query('select * from material_brewing', con=engine)
material_brewing_df

,id_material,material_type
0,1,Grain Products
1,2,Non-Grain Products


Merged the datasets now with the id's to the original dataframe, and deleted the columns that were not needed anymore.

In [14]:
# merge the dataframes

merged_brewing_materials = pd.merge(brewing_materials_df, types_brewing_df, how='left', on='type')
merged_brewing_materials = pd.merge(merged_brewing_materials, material_brewing_df, how='left', on='material_type')
merged_brewing_materials = merged_brewing_materials[["year", "month", "month_current", "id_types", "id_material"]] 

merged_brewing_materials

,year,month,month_current,id_types,id_material
0,2008,1,374165152,1,1
1,2008,1,57563519,2,1
2,2008,1,72402143,3,1
3,2008,1,3800844,4,1
4,2008,1,1177186,5,1
5,2008,1,78358212,6,2
6,2008,1,4506546,7,2
7,2008,1,621912,8,2
8,2008,1,1291615,9,2
9,2008,2,355687578,1,1


In [15]:
presentations_df = pd.read_sql_query('select * from presentation', con=engine)
presentations_df

,id_presentation,presentation
0,1,On Premises
1,2,Bottles and Cans
2,3,Kegs and Barrels


In [16]:
merged_beer_states = pd.merge(beer_states_df, presentations_df, how='left', on='presentation')
merged_beer_states = merged_beer_states[["state", "year", "barrels", "id_presentation"]]
merged_beer_states

,state,year,barrels,id_presentation
0,AK,2008,2067.69,1
1,AK,2009,2263.65,1
2,AK,2010,1929.15,1
3,AK,2011,2251.02,1
4,AK,2012,2312.43,1
5,AK,2013,2155.60,1
6,AK,2014,2114.97,1
7,AK,2015,4511.48,1
8,AK,2016,13315.93,1
9,AK,2017,15974.10,1


Connected the merged dataframes to the data base.

In [17]:
merged_brewing_materials.to_sql(name="merged_brewing_materials", con=engine, if_exists="append", index=False)

In [18]:
merged_beer_states.to_sql(name="merged_beer_states", con=engine, if_exists="append", index=False)

Joined the two merged dataframes to get the final table with all the information in it.

In [22]:
merged_brewing_materials = pd.read_sql_query('SELECT merged_brewing_materials.year, merged_brewing_materials.month, merged_brewing_materials.month_current, merged_brewing_materials.id_types, merged_brewing_materials.id_material, merged_beer_states.state, merged_beer_states.id_presentation FROM merged_brewing_materials JOIN merged_beer_states ON merged_brewing_materials.year = merged_beer_states.year;', con=engine)
merged_brewing_materials

,year,month,month_current,id_types,id_material,state,id_presentation
0,2008,1,374165152,1,1,WY,3
1,2008,1,374165152,1,1,WV,3
2,2008,1,374165152,1,1,WI,3
3,2008,1,374165152,1,1,WA,3
4,2008,1,374165152,1,1,VT,3
5,2008,1,374165152,1,1,VA,3
6,2008,1,374165152,1,1,UT,3
7,2008,1,374165152,1,1,TX,3
8,2008,1,374165152,1,1,TN,3
9,2008,1,374165152,1,1,SD,3
